### Clustering Splitpea networks
Cluster the splitpea output and do enrichment analysis on the cluster labels

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
from karateclub.dataset import GraphSetReader
from karateclub import FeatherGraph
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from plotnine import *
from sklearn.cluster import SpectralClustering

In [2]:
# metastatic + unannotated samples to remove
SKIP_LIST = ["TCGA-AC-A6IX-06A-11R-A32P-07", "TCGA-BH-A18V-06A-11R-A213-07", "TCGA-BH-A1ES-06A-12R-A24H-07", 
             "TCGA-BH-A1FE-06A-11R-A213-07", "TCGA-E2-A15A-06A-11R-A12D-07", "TCGA-E2-A15E-06A-11R-A12D-07", 
             "TCGA-E2-A15K-06A-11R-A12P-07", "TCGA-HZ-A9TJ-06A-11R-A41B-07", "TCGA-IB-7647-01A-11R-2156-07", 
             "TCGA-A7-A26F-01A-21R-A169-07", "TCGA-A7-A13G-01A-11R-A13Q-07", "TCGA-A7-A26I-01A-11R-A169-07", 
             "TCGA-A8-A09C-01A-11R-A00Z-07", "TCGA-AC-A2QH-01A-11R-A18M-07", "TCGA-AC-A3OD-01A-11R-A21T-07", 
             "TCGA-AC-A3QQ-01A-11R-A22K-07", "TCGA-BH-A0BS-01A-11R-A12P-07", "TCGA-E2-A14S-01A-11R-A12D-07", 
             "TCGA-LD-A7W5-01A-22R-A352-07"]

In [27]:
# read in the splitpea networks
can = 'BRCA'
net_dir = '/IRIS/' + can + '-pickle/'
full_nets = []
neg_nets = []
pos_nets = []
sams = []
for fn in os.listdir(net_dir):
    if 'edges.pickle' in fn:
        if fn.replace('.edges.pickle', '') in SKIP_LIST:
            print('skipping ' + fn.replace('.edges.pickle', ''))
            continue
        print(fn)
        sams.append(fn.replace('.edges.pickle', ''))
        g = pickle.load(open(net_dir + fn, 'rb'))

        g_largest_cc = g.subgraph(max(nx.connected_components(g), key=len)).copy()    
        g_largest_cc_neg = g_largest_cc.copy()
        g_largest_cc_pos = g_largest_cc.copy()
        
        for gi, gj in g_largest_cc.edges():
            if g_largest_cc[gi][gj]['chaos']:
                g_largest_cc_neg.remove_edge(gi, gj)
                g_largest_cc_pos.remove_edge(gi, gj)
                continue

            if g_largest_cc_pos[gi][gj]['weight'] <= 0:
                g_largest_cc_pos.remove_edge(gi, gj)
                g_largest_cc_neg[gi][gj]['weight'] = - g_largest_cc_neg[gi][gj]['weight']
            else:
                g_largest_cc_neg.remove_edge(gi, gj)
        
        # pos and neg may not be fully connected anymore
        g_largest_cc_neg = g_largest_cc_neg.subgraph(max(nx.connected_components(g_largest_cc_neg), key=len))
        g_largest_cc_pos = g_largest_cc_pos.subgraph(max(nx.connected_components(g_largest_cc_pos), key=len))

        full_nets.append(g_largest_cc)
        neg_nets.append(g_largest_cc_neg)
        pos_nets.append(g_largest_cc_pos)

TCGA-A2-A0CR-01A-11R-A22K-07.edges.pickle
TCGA-A2-A0CS-01A-11R-A115-07.edges.pickle
TCGA-A2-A0CQ-01A-21R-A034-07.edges.pickle
TCGA-A1-A0SH-01A-11R-A084-07.edges.pickle
TCGA-A1-A0SI-01A-11R-A144-07.edges.pickle
TCGA-A1-A0SE-01A-11R-A084-07.edges.pickle
TCGA-A2-A04N-01A-11R-A115-07.edges.pickle
TCGA-A2-A04Y-01A-21R-A034-07.edges.pickle
TCGA-5T-A9QA-01A-11R-A41B-07.edges.pickle
TCGA-A1-A0SD-01A-11R-A115-07.edges.pickle
TCGA-A2-A04X-01A-21R-A034-07.edges.pickle
TCGA-A1-A0SF-01A-11R-A144-07.edges.pickle
TCGA-A2-A04Q-01A-21R-A034-07.edges.pickle
TCGA-A2-A04V-01A-21R-A034-07.edges.pickle
TCGA-A1-A0SM-01A-11R-A084-07.edges.pickle
TCGA-A1-A0SG-01A-11R-A144-07.edges.pickle
TCGA-5L-AAT1-01A-12R-A41B-07.edges.pickle
TCGA-A2-A0CL-01A-11R-A115-07.edges.pickle
TCGA-3C-AALK-01A-11R-A41B-07.edges.pickle
TCGA-A1-A0SJ-01A-11R-A084-07.edges.pickle
TCGA-A2-A04R-01A-41R-A109-07.edges.pickle
TCGA-A2-A0CT-01A-31R-A056-07.edges.pickle
TCGA-A2-A0CO-01A-13R-A22K-07.edges.pickle
TCGA-A1-A0SQ-01A-21R-A144-07.edges

In [32]:
# read in the splitpea networks
can = 'PAAD'
net_dir = '/IRIS/' + can + '-pickle/'
for fn in os.listdir(net_dir):
    if 'edges.pickle' in fn:
        print(fn)
        sams.append(fn.replace('.edges.pickle', ''))
        g = pickle.load(open(net_dir + fn, 'rb'))

        g_largest_cc = g.subgraph(max(nx.connected_components(g), key=len)).copy()    
        g_largest_cc_neg = g_largest_cc.copy()
        g_largest_cc_pos = g_largest_cc.copy()
        
        for gi, gj in g_largest_cc.edges():
            if g_largest_cc[gi][gj]['chaos']:
                g_largest_cc_neg.remove_edge(gi, gj)
                g_largest_cc_pos.remove_edge(gi, gj)
                continue

            if g_largest_cc_pos[gi][gj]['weight'] <= 0:
                g_largest_cc_pos.remove_edge(gi, gj)
                g_largest_cc_neg[gi][gj]['weight'] = - g_largest_cc_neg[gi][gj]['weight']
            else:
                g_largest_cc_neg.remove_edge(gi, gj)
        
        # pos and neg may not be fully connected anymore
        g_largest_cc_neg = g_largest_cc_neg.subgraph(max(nx.connected_components(g_largest_cc_neg), key=len))
        g_largest_cc_pos = g_largest_cc_pos.subgraph(max(nx.connected_components(g_largest_cc_pos), key=len))

        full_nets.append(g_largest_cc)
        neg_nets.append(g_largest_cc_neg)
        pos_nets.append(g_largest_cc_pos)

TCGA-2J-AAB9-01A-11R-A41B-07.edges.pickle
TCGA-2J-AABO-01A-21R-A41B-07.edges.pickle
TCGA-2J-AAB8-01A-12R-A41B-07.edges.pickle
TCGA-2J-AABE-01A-12R-A41B-07.edges.pickle
TCGA-2J-AABR-01A-11R-A41B-07.edges.pickle
TCGA-2J-AAB1-01A-11R-A41B-07.edges.pickle
TCGA-2J-AABK-01A-31R-A41B-07.edges.pickle
TCGA-2J-AABT-01A-11R-A41B-07.edges.pickle
TCGA-2J-AABA-01A-21R-A41B-07.edges.pickle
TCGA-2J-AABH-01A-21R-A41B-07.edges.pickle
TCGA-2J-AAB6-01A-11R-A41B-07.edges.pickle
TCGA-2J-AABP-01A-11R-A41B-07.edges.pickle
TCGA-2J-AAB4-01A-12R-A41B-07.edges.pickle
TCGA-2J-AABF-01A-31R-A41B-07.edges.pickle
TCGA-2J-AABI-01A-12R-A41B-07.edges.pickle
TCGA-2J-AABV-01A-12R-A41B-07.edges.pickle
TCGA-F2-6880-01A-11R-2156-07.edges.pickle
TCGA-3A-A9I5-01A-11R-A38C-07.edges.pickle
TCGA-2L-AAQL-01A-11R-A38C-07.edges.pickle
TCGA-2J-AABU-01A-11R-A41B-07.edges.pickle
TCGA-F2-A44H-01A-11R-A26U-07.edges.pickle
TCGA-FB-A78T-01A-12R-A32O-07.edges.pickle
TCGA-3A-A9I9-01A-11R-A38C-07.edges.pickle
TCGA-FB-A4P6-01A-12R-A26U-07.edges

In [33]:
# make array of networkx objects (graphs) that start with 0 index and are consecutive
graphs = []
for n in pos_nets:
    graphs.append(nx.convert_node_labels_to_integers(n, first_label=0, ordering='default', label_attribute='entrez'))

In [34]:
# get FEATHER embeddings for the graphs
model = FeatherGraph()
model.fit(graphs)
X = model.get_embedding()

X.shape

(1284, 500)

In [35]:
df_X = pd.DataFrame(X)
df_X['sam'] = sams
df_X

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,sam
0,0.402537,0.384059,0.338885,0.277437,0.212974,0.157579,0.118617,0.096781,0.086329,0.077381,...,0.000426,0.000422,0.000415,0.000403,0.000388,0.000369,0.000346,0.000321,0.000292,TCGA-A2-A0CR-01A-11R-A22K-07
1,0.354873,0.345930,0.322948,0.288373,0.245672,0.198718,0.151133,0.105726,0.064133,0.026729,...,0.000106,0.000106,0.000107,0.000106,0.000105,0.000102,0.000100,0.000096,0.000092,TCGA-A2-A0CS-01A-11R-A115-07
2,0.398444,0.380469,0.336427,0.276229,0.212508,0.156836,0.116376,0.092033,0.078650,0.067160,...,0.000622,0.000619,0.000610,0.000596,0.000576,0.000551,0.000522,0.000488,0.000450,TCGA-A2-A0CQ-01A-21R-A034-07
3,0.352778,0.344336,0.322566,0.289593,0.248463,0.202644,0.155475,0.109683,0.067040,0.028233,...,0.000110,0.000110,0.000109,0.000106,0.000103,0.000099,0.000095,0.000089,0.000083,TCGA-A1-A0SH-01A-11R-A084-07
4,0.357874,0.348715,0.325193,0.289844,0.246260,0.198448,0.150148,0.104247,0.062406,0.024977,...,0.000130,0.000131,0.000132,0.000132,0.000131,0.000130,0.000127,0.000124,0.000120,TCGA-A1-A0SI-01A-11R-A144-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,0.367567,0.357282,0.331179,0.292854,0.247280,0.199756,0.154823,0.115388,0.082246,0.054105,...,0.000040,0.000040,0.000040,0.000039,0.000039,0.000037,0.000036,0.000034,0.000032,TCGA-L1-A7W4-01A-12R-A36G-07
1280,0.348973,0.341878,0.323610,0.295994,0.261562,0.223035,0.182807,0.142537,0.102961,0.063950,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TCGA-YY-A8LH-01A-11R-A36G-07
1281,0.350501,0.343094,0.323960,0.294860,0.258295,0.217062,0.173790,0.130557,0.088646,0.048510,...,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,TCGA-Z5-AAPL-01A-12R-A41B-07
1282,0.344814,0.337685,0.319311,0.291490,0.256737,0.217803,0.177174,0.136669,0.097221,0.058886,...,0.000036,0.000036,0.000035,0.000034,0.000033,0.000032,0.000030,0.000029,0.000026,TCGA-YH-A8SY-01A-11R-A37L-07


In [36]:
df_X.to_csv('BRCA-PAAD-FEATHER-matrix.txt', sep="\t", index=False, quoting=False)